In [ ]:
import os
import subprocess
import numpy as np

In [ ]:
class FilesBuilder:
    def __init__(self, y_array, z_array, rho_array, freqs, rx_pos, rx_label=None, path=os.getcw1d()):
        self.path = path
        if y_array is None:
            raise KeyError("Please give a valid y array.")
        self.y_array = y_array
        if z_array is None:
            raise KeyError("Please give a valid z array.")
        self.z_array = z_array
        if rho_array is None:
            raise KeyError("Please give a valid rho array.")
        self.rho_array = rho_array
        if freqs is None:
            raise KeyError("No frequency given to file generator.")
        self.freqs = freqs
        if rx_pos is None:
            raise KeyError("No receiver position given to file generator.")
        self.rx_pos = rx_pos
        if rx_label is None:
            self.rx_label = rx_label
        
    def __call__(self, *args):
        if args[0] == "full":
            self.write_model(filename=args[1])
            self.write_observer(filename=args[2])
        elif args[0] == "model":
            self.write_model(filename=args[1])
        elif args[0] == "data_templet":
            self.write_observer(filename=args[1])
        
        subprocess.run(["Mod2DMT.exe", "-F", "model.mod", "data_templet.dat", "obs_data.dat"])

    def write_model(self, filename="model"):
        with open(os.path.join(self.path, filename+".mod"), "w") as file:
            file.write(str(len(self.y_array))+"\t"+str(len(self.z_array))+"\tLINEAR\n")
            file.write(np.array2string(self.y_array, max_line_width=104).replace('[', '').replace(']','')+"\n")
            file.write(np.array2string(self.z_array, max_line_width=104).replace('[', '').replace(']','')+"\n")
            file.write("0\n")
            for i in range(len(self.rho_array[0])):
                file.write(np.array2string(self.rho_array[:, i], max_line_width=245).replace('[', '').replace(']','')+"\n")

        print("Model File has been written to {}".format(str(self.path)))

    def write_observer(self, filename="data_templet"):
        with open(os.path.join(self.path, filename+".dat"), "w") as file:
            file.write("# This File is generated by Python 3.11\n")
            file.write("# Period(s) Code GG_Lat GG_Lon X(m) Y(m) Z(m) Component Real Imag Error\n")
            file.write("> "+"TE_Impedance\n")
            file.write("> "+"exp(-i\omega t)\n")
            file.write("> "+"[V/m]/[T]\n")
            file.write("> "+"0.00\n")
            file.write("> "+"0.000 0.000\n")
            file.write("> "+str(len(self.freqs))+str(len(self.rx_pos))+"\n")
            for i in self.freqs:
                for j in range(len(self.rx_pos)):
                    if self.rx_label is not None:
                        file.write(str(i)+"\t"+str(self.rx_label[j])+"\t"+"0.000\t0.000\t"
                                   +np.array2string(self.rx_pos[j], max_line_width=26).replace('[', '').replace(']','')+"\t"+"TE"+"\t0.000000E+00"*3+"\n")
                    else:
                        file.write(str(i)+"\t"+str(j)+"\t"+"0.000\t0.000\t"
                                   +np.array2string(self.rx_pos[j], precision=3).replace('[', '').replace(']','')+"\t"+"TE"+"\t0.000000E+00"*3+"\n")
            
            print("Data Templet File has been written to {}".format(str(self.path)))
                        
    

In [ ]:
y_array = np.array([2000]*120)
z_array = np.array([500]*20)
rho_array = np.array([np.log10(1000)]*len(y_array)*len(z_array)).reshape((len(y_array), len(z_array)))
freqs = np.logspace(-1, 3, 20)
rx_pos = np.zeros((40, 3))
rx_pos[:, 1] = np.linspace(60000, 180000, 40)

filemaker = FilesBuilder(y_array=y_array, z_array=z_array, rho_array=rho_array, freqs=freqs, rx_pos=rx_pos)
filemaker("full", "model", "data")